In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
import numpy as np

2025-09-13 00:59:51.753088: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(
    project="Experiments Series 1.01",
    config={
        "Layer1": 256,
        "Activation_1": "relu",
        "Dropout1": "No dropout",
        "Layer2": 256,
        "Activation_2": "sigmoid",
        "Dropout2": "No dropout",
        "Layer3": 10,
        "Activation_3": "softmax",
        "Dropout3": "No dropout",
        "Optimizer": "adam",
        "Metric": "accuracy",
        "Epoch": 30,
        "Batch_size": 10,
        "Eta": 1e-5,
        "L1": 1e-4,
        "Loss": "binary_crossentropy"
    }
)

In [4]:
config = wandb.config

In [5]:
ds=mnist.load_data()

In [6]:
(x_train,y_train),(x_test,y_test)=ds
print(x_train.shape,x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [7]:
# Se hace al aplanado de las imágenes
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# Se modifica el tipo de datos a tipo flotante (para evitar conflictos)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Se normalizan los datos acorde a la intensidad de los píxeles
x_train /= 255.
x_test /= 255.

In [8]:
# Se realiza el one-hot enconding:

# Se definen el número de clases que hay en la base de datos
classes = config.Layer3

# Se realiza la conversión:
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [9]:
# Se configuran los hiperparámetros del modelo:

# Valor del Learning Rate
eta = config.Eta
eta = float(eta) # Tiene que ser un dato de tipo flotante, pues de lo contrario
                 # tiene conflicto

# Número de épocas
epochs = config.Epoch

# Tamaño del mini-batch
mini_batch= config.Batch_size

In [10]:
# Se realiza la creación del modelo a como se había definido usando Numpy:
model = Sequential([
    Dense(config.Layer1, activation = config.Activation_1, input_shape = (784,), kernel_regularizer = regularizers.L1(config.L1)),
    Dense(config.Layer2, activation = config.Activation_2, kernel_regularizer = regularizers.L1(config.L1)),
    Dense(classes, activation = config.Activation_3, kernel_regularizer = regularizers.L1(config.L1))
])
model.summary()
# El modelo es secuencial, se van uniendo neurona por neurona de forma consecutiva

2025-09-13 00:59:55.995124: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2025-09-13 00:59:55.995172: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-09-13 00:59:55.995180: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: emma
2025-09-13 00:59:55.995184: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] hostname: emma
2025-09-13 00:59:55.995554: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] libcuda reported version is: 470.256.2
2025-09-13 00:59:55.995599: I external/local_xla/xla/stream_executor/cuda/cuda_diag

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(loss=config.Loss, optimizer = config.Optimizer,
              metrics=[config.Metric])

history = model.fit(x_train, y_train,
                    batch_size = mini_batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test),
                    callbacks=[WandbMetricsLogger(log_freq=5),
                               WandbModelCheckpoint("models.keras")]
                    )

2025-09-13 00:59:56.173587: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8043 - loss: 0.2867 - val_accuracy: 0.9001 - val_loss: 0.1477
Epoch 2/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9038 - loss: 0.1426 - val_accuracy: 0.9187 - val_loss: 0.1271
Epoch 3/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9133 - loss: 0.1291 - val_accuracy: 0.9206 - val_loss: 0.1191
Epoch 4/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9156 - loss: 0.1229 - val_accuracy: 0.9250 - val_loss: 0.1148
Epoch 5/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9199 - loss: 0.1182 - val_accuracy: 0.9274 - val_loss: 0.1128
Epoch 6/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9203 - loss: 0.1150 - val_accuracy: 0.9281 - val_loss: 0.1104
Epoch 7/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9232 - loss: 0.1125 - val_accuracy: 0.9277 - val_loss: 0.1085
Epoch 8/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9253 - loss: 0

In [12]:
wandb.finish()

batch/accuracy,▁▅▅▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇███▇█████████████
batch/batch_step,▁▁▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▆▆▆▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▅▆▆▇▇▇▇▇▇▇█▇█████████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▄▅▅▆▆▆▇▅▇▇▆▇▆▆█▆█▇▇▆▇▇▇▇▇▇█▇▇
epoch/val_loss,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁
batch/accuracy,0.9323


In [13]:
mod = model.predict(x_test)

print("Resultado predicho: ")
print(np.where(mod[6] == np.max(mod[6])))
print("Resultado correcto: ")
print(np.where(y_test[6] == np.max(y_test[6])))


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step
Resultado predicho: 
(array([4]),)
Resultado correcto: 
(array([4]),)
